# Laserscanning - Exercise 2

#### Please upload the implemented solutions till <u>15.11.2022</u> to the studip folder of your group. The file should follow this format:
##### EX02_Group_XX.ipynb (e.g. EX02_Group_04.ipynb)
(In case you need to upload additional files, please also upload them using the same schema, e.g. EX02_Group_04.zip)

# Transformation
In this assignment the main topic is the estimation of a transformation between two point clouds(`left` and `right`). You will need the lecture scripts No. 3 “Rotations” and No. 4 “Estimation of transformations” (download: Stud.IP) to complete this assignment.

$${\Large r = sRl + t }$$
- $l$: 3D points left point cloud, $l\in\mathbb{R}^3$
- $r$: 3D point right point cloud, $r\in\mathbb{R}^3$
- $R$: $3\times3$ rotation matrix
- $t$: 3D translation vector, $t\in\mathbb{R}^3$
- $s$: scale factor, $s\in\mathbb{R}$

## Transformation Estimation
We use the following algorithm (or recipe from the lecture) to estimate the similarity transformation from point correspondences. Given are two point lists with the same number of points $l_i,r_i$ with  $i\in1,2,...,n$.

Our goal is to estimate $R,s$ and $t$. Therefore, we define an optimization problem where we need to find parameters which minimize the distance between those points:

$${\Large\sum_{i}{||(sRl_i + t) -r_i||}^{2} \stackrel{!}{=} min }$$


Recipe Steps:

    1) Reduce both point lists by their center of mass
    2) Compute the rotation
    3) Compute the scale
    4) Compute the translation

### 1) Reduction my the center of mass
Compute the center of mass as following (try not to use loops):
$${\Large l_{com}=\frac{1}{n}\sum_{i}l_i}$$
$${\Large r_{com}=\frac{1}{n}\sum_{i}r_i}$$
Afterwards, reduce all points by the calculatied center of mass:
$${\Large l_i' = l_i - l_{com}}$$
$${\Large _i' = r_i - r_{com}}$$


In [205]:
from plyfile import PlyData
import numpy as np

from tools import rotmat_from_quat # function from last exercise
#You can use the predefined function as follows:  R = rotmat_from_quat(q) or: use your own

In [206]:
import numpy as np
def com(points): 
    # calculate the center of mass of the points
    ### BEGIN SOLUTION
    
    c_m_points=np.mean(points,axis=0)
    


    return c_m_points
    ### END SOLUTION
    
    #raise NotImplementedError()

<u>Please explain your code here:</u>

SOME SENTENCES
The center of mass of the points is the average of all points. So we use mean() to calculate the average of right-points and left-points.

In [207]:
# define a homogeneous transformation for testing
tp = np.array([( 1, 0, 0), 
               (-1, 0, 0),
               ( 0, 1, 0), 
               ( 0,-1, 0),
               ( 0, 0, 1), 
               ( 0, 0, -1)
               ])
twos = np.array((2,2,2))
threes = np.array((3,-3,3))

In the following cell you can test your implementation and can compare it to correct resulting values.

In [208]:
assert np.array_equal(com(tp + twos), twos)
assert np.array_equal(com(tp + threes), threes)

### 2) Compute Rotation
Estimate the rotation quaternion $q$ as follows (compare lecture 4):

- Calculate the sum functions  $S_{xx}=\sum_{i}x_{l,i}'x_{r,i}'$, $S_{xy}=\sum_{i}x_{l,i}'y_{r,i}'$ ...  Generalized: $S_{<ld><rd>}=\sum_{i}<ld>_{l,i}'<rd>_{r,i}'$. $ld$ is left dimension and $rd$ right dimension.

- Compose matrix __$S$__ from the above $S_{<ld><rd>}$.

$$ S = \begin{bmatrix}
    S_{xx}+S_{yy}+S_{zz}& S_{yz}-S_{zy}        & S_{zx}-S_{xz}         & S_{xy}-S_{yx}  \\
    \cdotp              & S_{xx}-S_{yy}-S_{zz} & S_{xy}+S_{yx}         & S_{zx}+S_{xz}  \\
    \cdotp              & \cdotp               & -S_{xx}+S_{yy}-S_{zz} & S_{yz}+S_{zy}  \\
    \cdotp              & \cdotp               & \cdotp                & -S_{xx}-S_{yy}+S_{zz}
\end{bmatrix}$$

- Eigenvector of the larges eigenvalue of S is the searched rotation quternion $q$.
- The $3\times3$ rotation Matrix R is computed from q as discribed in lecture 3 slides 50/51. (This function is given by `rotmat_from_quat(q)`) 



In [209]:
# implement the sum function as the first step of the overall goal to compute the rotation
from numpy import *;
import numpy as np;
def sum_S(left_list, right_list):
    ### BEGIN SOLUTION
    
    s_x=left_list[0]*right_list[0]
    s_y=left_list[1]*right_list[1]
    s_z=left_list[2]*right_list[2]
    b=s_x+s_y+s_z

    return b
    
    
    ### END SOLUTION
    
    #raise NotImplementedError()

<u>Please explain your code here:</u>

x, y, and z in the two matrices left_list and right_list, are multiplied together and summed to get the result.

In the following cell you can test your implementation and can compare it to correct resulting values.

In [210]:
assert sum_S(np.array([1,2,1]),np.array([1,4,3])) == 12
assert sum_S(np.array([1,2,1]),np.array([1,1,3])) == 6

In [211]:
# implement the function to compose the S matrix as the second step of the overall goal to compute the rotation
def compose_S(left, right):
    ### BEGIN SOLUTION
    s_xx=0
    s_yy=0
    s_zz=0
    s_xy=0
    s_xz=0
    s_yx=0
    s_yz=0
    s_zx=0
    s_zy=0
    for i in range(len(left)):
        m_xx=left[i][0]*right[i][0]
        m_yy=left[i][1]*right[i][1]
        m_zz=left[i][2]*right[i][2]
        m_xy=left[i][0]*right[i][1]
        m_xz=left[i][0]*right[i][2]
        m_yx=left[i][1]*right[i][0]
        m_yz=left[i][1]*right[i][2]
        m_zx=left[i][2]*right[i][0]
        m_zy=left[i][2]*right[i][1]
        
        s_xx+=m_xx
        s_yy+=m_yy
        s_zz+=m_zz
        s_xy+=m_xy
        s_xz+=m_xz
        s_yx+=m_yx
        s_yz+=m_yz
        s_zx+=m_zx
        s_zy+=m_zy
        pass

    S=array([[s_xx+s_yy+s_zz,s_yz-s_zy,s_zx-s_xz,s_xy-s_yx],
        [s_yz-s_zy,s_xx-s_yy-s_zz,s_xy+s_yx,s_zx+s_xz],
        [s_zx-s_xz,s_xy+s_yx,-s_xx+s_yy-s_zz,s_yz+s_zy],
        [s_xy-s_yx,s_zx+s_xz,s_yz+s_zy,-s_xx-s_yy+s_zz]])
    return S

    ### END SOLUTION
    
    #raise NotImplementedError()

In [212]:
# this function derives the quaternion rotation from the S matrix using the singular value decomposition
def q_from_S(S):
    # take a look at numpy.linalg.eig() function for this task
    ### BEGIN SOLUTION
    
    c = np.linalg.eig(S)
    
    ew=c[0]
    l=list(ew)  
    m=max(l)
    num=l.index(m)
    d=c[1]
    q=d[:,num]

    return q    
    ### END SOLUTION
    
    #raise NotImplementedError()

<u>Please explain your code here:</u>

First we should calculate the sum functions  𝑆𝑥𝑥=∑𝑖𝑥′𝑙,𝑖𝑥′𝑟,𝑖 ,  𝑆𝑥𝑦=∑𝑖𝑥′𝑙,𝑖𝑦′𝑟,𝑖  ... 
Then we bring the previously calculated values into the formula for S.
We use numpy.linalg.eig() to calculate the eigenvalues and eigenvectors. Then we find the location of the maximum value in the eigenvalues. The eigenvector corresponding to this position is the quaternion we require.

In the following cell you can test your implementation and can compare it to correct resulting values.

In [213]:
tp = np.array([[ 1, 0, 0], 
               [-1, 0, 0],
               [ 0, 1, 0], 
               [ 0,-1, 0],
               [ 0, 0, 1], 
               [ 0, 0, -1]
               ])
rot_z_90 = np.array([[ 0,   1.0, 0  ],
                     [-1.0, 0,   0  ],
                     [ 0,   0,   1.0]
                    ])
rot_x_90 = np.array([[ 1,  0,  0],
                                [ 0,  0,  1],
                                [ 0, -1,  0]])
rot_y_90 = np.array([[ 0,  0,  1],
                                [ 0,  1,  0],
                                [ -1,  0, 0]])
tp_rot = np.dot(rot_z_90, np.transpose(tp)).transpose()
print("Sz90")
print(compose_S(tp, tp_rot))
q = q_from_S(compose_S(tp, tp_rot))
assert np.allclose(q, np.array([ 0.70710678,  0.,          0.,         -0.70710678]))
tp_rot = np.dot(rot_y_90, np.transpose(tp)).transpose()
q = q_from_S(compose_S(tp, tp_rot))
assert np.allclose(q, np.array([ 0.70710678,  0.,          0.70710678,  0.,     ]))
tp_rot = np.dot(rot_x_90, np.transpose(tp)).transpose()
q = q_from_S(compose_S(tp, tp_rot))
assert  np.allclose(q, np.array([ 0.70710678, -0.70710678,  0.,          0.,        ]))

Sz90
[[ 2.  0.  0. -4.]
 [ 0. -2.  0.  0.]
 [ 0.  0. -2.  0.]
 [-4.  0.  0.  2.]]


### 3) Compute scale
Compute the scale with the following formula:
$${\Large s = \sqrt{\frac{\sum_{i}||r_i'||^{2}}{\sum_{i}||l_i'||^{2}}}}$$

In [214]:
def calc_scale(left, right):
    ### BEGIN SOLUTION
    import math as m
    r_prime=[]
    l_prime=[]
    l_com=np.mean(left,axis=0)
    r_com=np.mean(right,axis=0)
    for i in right:
        r_prime.append(np.power(np.linalg.norm(i-r_com),2))
    for j in left:
        l_prime.append(np.power(np.linalg.norm(j-l_com),2))
    r_sum=np.sum(r_prime)
    l_sum=np.sum(l_prime)
    s=m.sqrt(r_sum/l_sum)

    return s
    ### END SOLUTION
    
   # raise NotImplementedError()

<u>Please explain your code here:</u>

SOME SENTENCES
We use np.linalg.norm() to calculate  one of the vector norms. Power is used to raise the number on the left to the power of the exponent of the right. We compute the scale with the above formula.

In the following cell you can test your implementation and can compare it to correct resulting values.

In [215]:
assert calc_scale(tp, tp*1) == 1.0
assert calc_scale(tp, tp*2) == 2.0
assert calc_scale(tp, tp*4) == 4.0

### 4) Compute Translation
The translation is computed with known scale $s$ and rotation $R$.
$${\Large t=r_{com} - sRl_{com}}$$

In [216]:
def calc_t(l_com, r_com, s, R):
    ### BEGIN SOLUTION
    import numpy as np
    
    t=r_com-s*np.dot(R,l_com)
    

    return t
    
    ### END SOLUTION

<u>Please explain your code here:</u>

SOME SENTENCES

In the following cell you can test your implementation and can compare it to correct resulting values.

In [217]:
from tools import rotmat_from_quat
rot_y_90 = np.array([[ 0,  0,  1],
                     [ 0,  1,  0],
                     [ -1,  0, 0]])
tp_rot = np.dot(rot_y_90, np.transpose(tp)).transpose()
tp_rot = tp_rot + np.array([10, 5, 2])
q = q_from_S(compose_S(tp, tp_rot))
R = rotmat_from_quat(q)
s = calc_scale(tp, tp_rot)
assert np.allclose(calc_t(com(tp), com(tp_rot),s,R), np.array([ 10.,   5.,   2.]))

## Application
Please estimate the transformation between point cloud left and right, given in `.ply` format. Apply the transformation to the point cloud in order to reproduce the second cloud. Calculate the mean distance between the second cloud and the reproduced cloud.

In [218]:
# load the data
from plyfile import PlyData
_data = PlyData.read('points_left.ply').elements[0].data
left = np.stack((_data['x'], _data['y'], _data['z']), axis=1)
_data = PlyData.read('points_right.ply').elements[0].data
right = np.stack((_data['x'], _data['y'], _data['z']), axis=1) 

print(left,'l')
print(right,'r')

[[10.019  2.656  0.885]
 [ 7.559  4.033  0.896]
 [ 8.175  4.201  0.911]
 ...
 [ 2.55  15.143 11.826]
 [ 2.73  15.114 11.951]
 [ 2.632 15.21  11.929]] l
[[10.418883  5.416085  0.972391]
 [ 8.675199  4.396667  2.939166]
 [ 9.235649  4.702922  2.942009]
 ...
 [12.440327 -6.050413 14.967026]
 [12.6469   -6.075485 14.892427]
 [12.575439 -6.09134  15.01052 ]] r


Estimate the rotation of the two given point clouds, `left`and `right`. Furthermore, calculate the error of the original `right`and rotated `left` point cloud.

In [219]:
### BEGIN SOLUTION
import math as m
import numpy as np
from tools import rotmat_from_quat
l_com=np.mean(left,axis=0)
r_com=np.mean(right,axis=0)

S=compose_S(left, right)
q=q_from_S(S)
s=calc_scale(left, right)
R = rotmat_from_quat(q)
t=calc_t(l_com, r_com, s, R)

re=np.dot(left,R)
error=np.mean(re-right,axis=0)

print(R)

print(error)



### END SOLUTION

#raise NotImplementedError()

[[ 0.81260534  0.19218197  0.55021692]
 [ 0.52362426  0.17379355 -0.83403443]
 [-0.25591053  0.96584776  0.04059442]]
[ -3.40842351   9.03725705 -10.60066022]


<u>Please explain your code here:</u>
At first we calculate the R through left and right. And then calculate the reproduced cloud through np.dot(left,R). At last calculate the mean value of error through np.mean(re-right,axis)

SOME SENTENCES